<a href="https://colab.research.google.com/github/Brandon12231/780/blob/main/Example_GlassMaking_Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Glass Maker's Problem

In the following code, we'll implement the glass maker's problem in Pyomo. The model is:

By changing the

**Decision variables**

$x_1$ = 100s of cases of six ounce glasses to produce and

$x_2$ = 100s of cases of ten ounce glasses to produce

**Objective**

Maximize revenue
$$
z = 500 x_1 + 450 x_2
$$

**Constraint**

Subject to

$$
\begin{align}
6 x_1   & + & 5 x_2   & \le 60  & \text{(hours)}\\
10 x_1  & + & 20 x_2  & \le 150 & \text{(storage)}\\
x_1     &   &         & \le 8  & \text{(demand)}
\end{align}
$$
constraints.


Install Pyomo and a few optimization engines.

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre    # Installs a set of good open source optimization tools
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'    # Add to path so we can find it

import pyomo.environ as pyo         # Import pyomo

Instantiate an engine that we can use to optimize.

In [ ]:
opt = pyo.SolverFactory('cbc')
  # Instantiate the 'CBC' optimization engine: one of the best open source engines that can solve
  # linear and non-linear problems with continuous and intger decision variables.

The next step is to create an optimization model in Pyomo.

In [ ]:
m = pyo.ConcreteModel()
  # Instantiate a model

# Define variables
m.x1 = pyo.Var(domain=pyo.NonNegativeReals)
m.x2 = pyo.Var(domain=pyo.NonNegativeReals)
  # Two variables, defined to be continuous and non-negative.

m.RevenueObj = pyo.Objective( expr = 500*m.x1 + 450*m.x2, sense=pyo.maximize )

m.HoursConstraint = pyo.Constraint( expr = 6*m.x1 + 5*m.x2 <= 60 )
m.StorageConstraint = pyo.Constraint( expr = 10*m.x1 + 20*m.x2 <= 150 )
m.DemandConstraint = pyo.Constraint( expr = m.x1 <= 8 )
  # You could have read the objective and the constraint coefficients from a file and used them in formulating these equations.

We can print the model to get a sense of what is in it:

In [ ]:
m.pprint()

It has a lot more information than we care about right now. We note that it has the two variables, objective, and three constraints that we defined. The variables haven't been set by teh optimizer, so they have value 'None'.

In [ ]:
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
  # Only necessary to access the shadow price here. Must be done before optimizing.
  # The Suffix is the mechanism through which Pyomo accesses the internal variables.
  # IMPORT here suggests that we are going to use the variable to receive some information from the
  # optimizer (in contrast to send some extra information to it).

opt.solve(m)
# There are two ways of accessing the optimum values:
print("Decision variables: ")
print("Six ounce glasses  ", m.x1.value)  # .value
print("Ten ounce glasses  ", m.x2())      # or "calling" it with ()

print("\nObjective: ")
print("Revenue obtained", m.RevenueObj())

# We can also get the LHS of the constraints, showing us how much of the resources were used:
print("\nResources used: ")
print("Hours used   ", m.HoursConstraint())
print("Storage used   ", m.StorageConstraint())
print("Demand used   ", m.DemandConstraint())

# Because we told the optimizer that we want the "dual" variables after optimization these are available to us:
print("\nShadow prices of the ")
print("hour limit    = ", m.dual[m.HoursConstraint])
print("storage limit = ", m.dual[m.StorageConstraint])
print("demand limit  = ", m.dual[m.DemandConstraint])


## Extra Notes

This highlights the features that you need to complete the HW1. There are a ton more. You can learn them, as necessary, from the resources at the end.

### Abstract vs Concrete Models

There are two types of models in Pyomo: concrete and abstract. The former can be created when all your data elements (constraint parameters and objective coefficients) are available at the time of model creation. The abstract model allows us to create a model without "hard coding" data as part of it. The former is easier (hence we use it here). The latter allows us to separately specify a data file whose content will decide the model details. Abstract model is more common in some of the more popular optimization modeling languages (such as AMPL), as it allows us to focus on the type of model, without getting bogged down in the details of a specific instance.

Having said that, because Pyomo is being used within Python, we could achieve quite a bit of "program--data separation" using a concrete model too. We use ConcreteModel in this example since it's a little easier to use.

### Further Resources

1. A nice [tutorial style cookbook](https://pyomo.readthedocs.io/en/stable/) by Professor Jeffrey Kantor from Notre Dame.
1. The [official Pyomo documentation](https://pyomo.readthedocs.io/en/stable/).
